In [19]:
import requests
from bs4 import BeautifulSoup

urls = {
    "BIZNES" : {
        "url" : "https://businessinsider.com.pl/biznes",
        "pages" : 10
        }
}


def get_page_data(url):
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            html_content = response.text
            print("Pobrano zawartość strony pomyślnie.")
        else:
            print(f"Błąd podczas pobierania strony. Kod statusu: {response.status_code}")
            exit()

    except requests.exceptions.RequestException as e:
        print(f"Wystąpił błąd połączenia: {e}")
        exit()

def get_business_main_data(url,page):
    if page > 1:
        url = url + f'?page={page}'
        
    content = get_page_data(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    main = soup.find('main', class_='main')
    return main

def extract_headers(element, header, class_name=''):
    return element.find_all(header,class_= class_name)

def extract_text(elements):
    text = []
    for element in elements:
        text.append(element.get_text(strip=True))
    return text

def get_business_headers(url,pages):
    text_headers = []
    for page in range(1,pages+1):
        main_content = get_business_main_data(url,page)
        headers = extract_headers(main_content,'h2','item_title')
        text_headers.extend(extract_text(headers))
    return text_headers


categories_headers = {
    "BIZNES" : []
}

categories_headers['BIZNES'] = get_business_headers(urls['BIZNES']['url'],urls['BIZNES']['pages'])
print(categories_headers['BIZNES'])

Pobrano zawartość strony pomyślnie.
Pobrano zawartość strony pomyślnie.
Pobrano zawartość strony pomyślnie.
Pobrano zawartość strony pomyślnie.
Pobrano zawartość strony pomyślnie.
Pobrano zawartość strony pomyślnie.
Pobrano zawartość strony pomyślnie.
Pobrano zawartość strony pomyślnie.
Pobrano zawartość strony pomyślnie.
Pobrano zawartość strony pomyślnie.
['NEXT prezentuje limitowaną ofertę noworoczną', 'Black Friday przestał być tylko świętem okazji. Stał się testem technologicznej dojrzałości rynku', 'Łódzcy naukowcy wspierają pacjentów z cukrzycą', 'Przyszłość handlu napędzana przez ludzi, technologię i odporność', 'Michał Sołowow ma nowego partnera w atomowym biznesie. To znany na świecie gigant', 'Jak z polskiej wsi dojść na globalny rynek. Trzech ekspertów o tym, jak naprawdę buduje się firmy światowej klasy', 'Urząd Transportu Kolejowego grozi przewoźnikowi z Czech. Zaczęła się konkurencja na kolei i może jej zaraz nie być', 'mBank jest kobietą!', 'Cinkciarz.pl w upadłości. Sy